In [1]:
import geopandas as gpd
import pandas as pd
import geojson
import numpy as np
import folium
from shapely.geometry import shape, Point
from shapely import geometry

In [2]:

ray = pd.read_excel('ray.xlsx')


In [3]:
#CAMBIAR 1000 por XXXX
ray = ray[['FECHA','LATITUD','LONGITUD','CORRIENTE']]

In [4]:
ray

,FECHA,LATITUD,LONGITUD,CORRIENTE
0,2018-08-01,5.3726,-74.5058,-8.9
1,2018-08-01,4.8297,-73.5334,-31.3
2,2018-08-01,4.7931,-73.4938,-50.9
3,2018-08-01,4.9744,-73.9611,-6.7
4,2018-08-01,4.7014,-73.3774,-18.1
...,...,...,...,...
994,2018-08-02,5.6766,-74.4469,-6.9
995,2018-08-02,5.6393,-74.3939,5.5
996,2018-08-02,5.6534,-74.5210,5.7
997,2018-08-02,5.6822,-74.5466,6.8


In [5]:
df = gpd.read_file('dataframe.geojson')
df = df[['EQUIPO', 'geometry']]

ray = gpd.GeoDataFrame(ray, geometry = gpd.points_from_xy(ray.LONGITUD, ray.LATITUD))
ray['FECHA'] = ray['FECHA'].dt.strftime('%Y%m%d')

In [6]:
ray['ubicacion'] = "otro"
r = 1/(1e2) # 1/1e3 es equivalente a 111 metros

l = list()
for i in range(0, len(df)):
    x = df.loc[i, 'geometry'].x
    y = df.loc[i, 'geometry'].y
    poly = shape(geometry.Polygon([[x-r, y-r],[x-r, y+r],[x+r, y+r], [x+r, y-r]]))
    l.extend([[df.loc[i, 'EQUIPO']]])
    for j in range(0, len(ray)):
        if poly.contains(ray.loc[j, 'geometry']):
            l[i].append(ray.loc[j,'CORRIENTE'])
            ray.at[j,'ubicacion'] = df.loc[i, 'EQUIPO']

In [7]:
ray['ubicacion'].value_counts()

otro       908
ERC5278     18
ERC5005     11
ERC5279      8
ESZ5072      6
ESZ5070      4
ERC5188      3
RC1881       3
SZ6070       2
SZ4443       2
SZ2542       1
SZ2412       1
RC2177       1
SZ3374       1
SZ6852       1
SZ2576       1
SZ1719       1
SZ4788       1
SZ6360       1
SZ2524       1
ERC5134      1
SZ4716       1
SZ3863       1
SZ7281       1
RC2051       1
SZ3292       1
SZ6451       1
RC2049       1
RC2201       1
SZ7019       1
SZ1679       1
RC840        1
RC509        1
SZ1569       1
ESZ5076      1
SZ6827       1
RC1454       1
SZ4961       1
SZ2210       1
SZ2509       1
SZ4530       1
RC1126       1
SZ5035       1
RC1337       1
Name: ubicacion, dtype: int64

In [8]:
map = folium.Map(location=[4.5, -74], tiles="Cartodb Positron", zoom_start=7)

ray = ray.set_crs("EPSG:4326")
ray = ray.set_crs(epsg=4326)

folium.GeoJson(ray
              ).add_to(map)


In [9]:
l = list()
l.extend([[df.loc[i, 'EQUIPO']]])
l[0].append(4)
l

[['ERC2150', 4]]

In [10]:
[x for x in l if len(x) > 1]

[['ERC2150', 4]]